In [1]:
#Import required libraries
import pandas as pd
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import BatchNormalization,GlobalAveragePooling2D,Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
import time

In [2]:
img_height, img_width=(225,225)
batch_size=32
#train,validation and test dataset
train_data_dir="/home/qwevo/CONDAPROJECTS/ip102_v1.1/ip102_v1.1/proccesed/images/train"
test_data_dir="/home/qwevo/CONDAPROJECTS/ip102_v1.1/ip102_v1.1/proccesed/images/test"
valid_data_dir="/home/qwevo/CONDAPROJECTS/ip102_v1.1/ip102_v1.1/proccesed/images/val"

In [3]:
#Keras ImageDataGenerator class makes image augmentation and labeling much easier
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.4)
#create the training dataset
train_generator =train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')
#creating validation dataset
valid_generator =train_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

Found 3240 images belonging to 18 classes.
Found 720 images belonging to 18 classes.


In [4]:
#creating the testdataset
test_generator =train_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height,img_width),
    batch_size=1,
    class_mode='categorical',
    subset='training')

Found 1080 images belonging to 18 classes.


In [5]:

x,y=test_generator.next()
x.shape

(1, 225, 225, 3)

In [6]:
x,y=train_generator.next()
x.shape

(32, 225, 225, 3)

In [7]:
#classes on the training set
train_generator.num_classes

18

In [8]:
#classes on the test set
test_generator.num_classes

18

In [9]:
#saving the labels 
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('labels.txt', 'w') as f:
 f.write(labels)

In [10]:
Name ="ip102res50={}".format(int(time.time()))
tensorboard = TensorBoard(log_dir="ip102res50logs/{}".format(Name))
base_model=ResNet50(include_top=False, weights='imagenet')
#base_model.load_weights('resnet50_weights_tf_dim_ordering_tf_kernels.h5')
x=base_model.output
#Add dense & Pooling Layer
x=GlobalAveragePooling2D()(x)
x=Dense(64,activation="relu")(x)
predictions=Dense(train_generator.num_classes,activation="sigmoid")(x)
model=Model(inputs=base_model.input,outputs=predictions)

for layer in base_model.layers:
    layer.trainable=False

#Compile the model    
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
#train the model for 5 epochs
model.fit(train_generator,epochs=5,validation_data = valid_generator,callbacks=[tensorboard])

Epoch 1/5
102/102 [==============================] - 605s 6s/step - loss: 1.8755 - accuracy: 0.4558 - val_loss: 0.9103 - val_accuracy: 0.7347
Epoch 2/5
102/102 [==============================] - 646s 6s/step - loss: 0.4439 - accuracy: 0.8658 - val_loss: 0.8521 - val_accuracy: 0.7486
Epoch 3/5
102/102 [==============================] - 11530s 114s/step - loss: 0.3016 - accuracy: 0.9111 - val_loss: 0.8426 - val_accuracy: 0.7458
Epoch 4/5
102/102 [==============================] - 478s 5s/step - loss: 0.2174 - accuracy: 0.9396 - val_loss: 0.7842 - val_accuracy: 0.7708
Epoch 5/5
102/102 [==============================] - 458s 4s/step - loss: 0.1870 - accuracy: 0.9460 - val_loss: 0.7235 - val_accuracy: 0.7736


In [11]:
print("Train Set loss, accuracy respectively: " + str(model.evaluate(train_generator)))

102/102 [==============================] - 372s 4s/step - loss: 0.1315 - accuracy: 0.9633
Train Set loss, accuracy respectively: [0.13148875534534454, 0.9632716178894043]


In [12]:
print("Validation Set loss, accuracy respectively: " + str(model.evaluate(valid_generator)))

23/23 [==============================] - 84s 4s/step - loss: 0.7222 - accuracy: 0.7778
Validation Set loss, accuracy respectively: [0.7222334146499634, 0.7777777910232544]


In [13]:
print("Test Set loss, accuracy respectively: " + str(model.evaluate(test_generator)))

1080/1080 [==============================] - 286s 264ms/step - loss: 0.3955 - accuracy: 0.8759
Test Set loss, accuracy respectively: [0.3954898416996002, 0.8759258985519409]


In [14]:
model.save('ip102resnet50.h5')

In [15]:
# Saving the model
import tensorflow as tf
saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir)


INFO:tensorflow:Assets written to: assets


In [16]:
#converting the model to tflitemodel
import tensorflow as tf
converter =tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()
with open('bugres50model.tflite', 'wb') as f:
 f.write(tflite_model)


In [17]:
#check the tflite model input details
import tensorflow as tf
interpreter=tf.lite.Interpreter(model_path="bugres50model.tflite")
input_details=interpreter.get_input_details()
output_details=interpreter.get_output_details()
print("input shape:",input_details[0]['shape'])
print("input type:",input_details[0]['dtype'])
print("output shape:",output_details[0]['shape'])
print("output type:",output_details[0]['dtype'])

input shape: [1 1 1 3]
input type: <class 'numpy.float32'>
output shape: [ 1 18]
output type: <class 'numpy.float32'>


In [18]:
#change the input 
interpreter.resize_tensor_input(input_details[0]['index'],(32,1,1,3))
interpreter.resize_tensor_input(output_details[0]['index'],(32,18))
interpreter.allocate_tensors()
input_details=interpreter.get_input_details()
output_details=interpreter.get_output_details()
print("input shape:",input_details[0]['shape'])
print("input type:",input_details[0]['dtype'])
print("output shape:",output_details[0]['shape'])
print("output type:",output_details[0]['dtype'])


input shape: [32  1  1  3]
input type: <class 'numpy.float32'>
output shape: [32 18]
output type: <class 'numpy.float32'>


In [19]:
test_generator.dtype

'float32'

In [20]:
x,y=test_generator.next()
x.dtype

dtype('float32')

In [21]:
#confusion matric
import pandas as pd
import seaborn as sn
import tensorflow as tf

model =tf.keras.models.load_model('try_pest_resnet50.h5')
filenames=test_generator.filenames
nb_samples=len(test_generator)
y_prob=[]
y_act=[]
test_generator.reset()
for _ in range(nb_samples):
    X_test,Y_test=test_generator.next()
    y_prob.append(model.predict(X_test))
    y_act.append(Y_test)

predicted_class=[list(train_generator.class_indices.keys())[i.argmax()]for i in y_prob]
actual_class=[list(train_generator.class_indices.keys())[i.argmax()]for i in y_act]
    
out_df=pd.DataFrame(np.vstack([predicted_class,actual_class]).T,columns=['predicted_class','actual_class'])
confusion_matrix=pd.crosstab(out_df['actual_class'],out_df['predicted_class'],rownames=['actual'],colnames=['predicted'])

sn.heatmap(confusion_matrix,cmap='Blues',annot=True,fmt='d')
plt.show()
print('test_accuracy : {}'.format((np.diagonal(confusion_matrix).sum()/confusion_matrix.sum().sum()*100)))

OSError: SavedModel file does not exist at: try_pest_resnet50.h5/{saved_model.pbtxt|saved_model.pb}

In [22]:
import tensorflow as tf
export_dir = 'saved_model/1'
tf.saved_model.save(model, export_dir)

INFO:tensorflow:Assets written to: saved_model/1/assets


INFO:tensorflow:Assets written to: saved_model/1/assets


In [23]:
#@title Select mode of optimization
mode = "Storage" #@param ["Default", "Storage", "Speed"]

if mode == 'Storage':
  optimization = tf.lite.Optimize.OPTIMIZE_FOR_SIZE
elif mode == 'Speed':
  optimization = tf.lite.Optimize.OPTIMIZE_FOR_LATENCY
else:
  optimization = tf.lite.Optimize.DEFAULT

In [24]:
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
#converter.optimizations = []
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
#converter.allow_custom_ops=False
tflite_model = converter.convert()

In [25]:
tflite_model_file = 'model.tflite'

with open(tflite_model_file, "wb") as f:
  f.write(tflite_model)

In [26]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

In [30]:
import cv2
img_array = cv2.imread('/home/qwevo/CONDAPROJECTS/ip102_v1.1/ip102_v1.1/proccesed/images/test/army worm')
img_array = cv2.resize(img_array,(225,225))
img_array = img_array/255
img_array = np.expand_dims(img_array,axis=0)
img = tf.cast(img_array, tf.float32)
img.shape

error: OpenCV(4.5.3) /tmp/pip-req-build-afu9cjzs/opencv/modules/imgproc/src/resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [ ]:
# Gather results for the randomly sampled test images

predictions = []

test_labels = []

test_images = []

interpreter.set_tensor(input_index, img)

interpreter.invoke()

predictions.append(interpreter.get_tensor(output_index))

#test_labels.append(label[0])

#test_images.append(np.array(img))

In [ ]:
predictions

In [ ]:
ind = np.argmax(predictions)
print('Predicted crop is: ',class_names[ind])